In [2]:
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers, losses, metrics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys

Using TensorFlow backend.


# 데이터 로드

In [3]:
text = open('D:/STUDY/dataset/novel/novel.txt',encoding='utf-8').read().lower()

In [4]:
text

 마찬가지야. 우리에게 중요한 건 산이지. 산은 변하지 않거든."\n "그런데 <일시적인 존재>란 뭐예요?"\n 한번 한 질문은 평생 포기해 본 적이 없는 어린 왕자가 다시 되물었다.\n "그건 <머지 않은 장례에 사라져 버릴 위험에 처해 있다>는 뜻이지."\n "내 꽃은 머지 않은 장래에 사라져 버릴 위험에 처해 있나요?"\n "물론이지."\n (내 꽃은 일시적 존재야. 세상에 대항할 무기라곤 네 개의 가시밖에 없고!그런데 나는 그 꽃을 내 별에 혼자 내버려두고 왔어!) 하고 어린 왕자는 생각했다.\n 그것은 후회스러운 느낌의 시작이었다. 그러나 그는 다시 용기를 냈다.\n "어디를 가보는 게 좋을까요?"\n 그가 물었다.\n "지구라는 별로 가봐. 대단히 이름높은 별이거든......"\n 그리하여 어린 왕자는 그의 꽃에 대해 생각하며 다시 길을 떠났다.\n 16\n 일곱 번째 별은 그래서 지구였다.\n 지구는 그저 그렇고 그런 보통 별이 아니었다! 그곳에는 1백 11명의 왕(물론 흑인나라의 왕을 포함해서)과 7천 명의 지리학자와 90만 명의 실업가. 7백 50만 명의 술주정뱅이, 3억 1천 1백만 명의 허영심 많은 사람들,즉 약20억쯤 되는 어른들이 살고 있다.전기가 발명되기 전까지는여섯 대륙을통틀어4십 6만 2천 5백 11명이나 되는가로등 점화하는 사람들을 두어야했다는 이야기를 들으면 여러분은 지구가 얼마나 큰지 짐작이 갈 것이다.\n 그래서 좀 멀리 떨어진 곳에서 보면눈부시게 멋진 광경이 벌어지는 것이었다. 그들이 무리지어 움직이는 모습은 오페라의 발레에서처럼 질서정연한것이었다.맨 처음은 뉴질랜드와 오스트레일리아의가로등 켜는 사람들의차례였다. 가로등을 켜고 나면 그들은 잠을 자러 갔다. 그러고 나면 중국과시베리아의 가로등 점화하는 사람들이 발레 무대에 나타났다. 그들 역시 무대 뒤로 살짝 몸을 감추고 나면러시아와 인도의 가로등 켜는 사람들이 나타나는 것이었다.그 다음번에는아프리카와 유럽의 가로등 켜는 사람들,또 그 다음에는 남아메리카의 가로등 켜는 사람

In [5]:
# 입력으로 들어가는 글자수
input_len = 30

# 입력 문장 리스트
input_sentence_list = []

# 출력 캐릭터 리스트
output_char_list = []

# 입력 문장과 바로 다음에 나오는 출력 캐릭터 데이터셋 생성
for i in range(0,len(text)-input_len):
    input_sentence_list.append(text[i:i+input_len])
    output_char_list.append(text[i+input_len])

In [6]:
# 로드한 텍스트 문서로 입력과 출력의 데이터셋을 만듭니다. 30글자씩 입력으로 하고 바로 그 다음 글자를 출력으로 합니다. 전체 텍스트에서 오른쪽으로 하나씩 이동하면서 계속 데이터 생성. 
# 이렇게 스스로 라벨을 만드는 머신러닝 방식을 자기지도학습이라고 합니다. 


In [7]:
# 입력 문장과 출력 캐릭터 출력
for i in range(0,10):
    print(input_sentence_list[i] + ' ->' +output_char_list[i])
    print()

﻿여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원 ->시

여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시 ->림

섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림 ->에

 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 -> 

살 적에 나는 "체험한 이야기"라는 제목의, 원시림에  ->관

 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관 ->한

적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 -> 

에 나는 "체험한 이야기"라는 제목의, 원시림에 관한  ->책

 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책 ->에

나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에 ->서



In [8]:
# 입력 문장 리스트 개수
# len(text)-input_len + 1 = 45772 - 30 + 1
len(input_sentence_list)

45743

In [9]:
chars = sorted(list(set(text)))
chars

['\n',
 ' ',
 '!',
 '"',
 '(',
 ')',
 ',',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '<',
 '>',
 '?',
 'b',
 '使',
 '儀',
 '商',
 '大',
 '小',
 '布',
 '式',
 '律',
 '惑',
 '星',
 '的',
 '紀',
 '紅',
 '詩',
 '隊',
 '가',
 '각',
 '간',
 '갈',
 '감',
 '갑',
 '값',
 '갔',
 '강',
 '갖',
 '같',
 '개',
 '객',
 '거',
 '걱',
 '건',
 '걷',
 '걸',
 '검',
 '겁',
 '것',
 '게',
 '겐',
 '겠',
 '겨',
 '겪',
 '견',
 '결',
 '겸',
 '겹',
 '겼',
 '경',
 '곁',
 '계',
 '고',
 '곡',
 '곤',
 '곧',
 '골',
 '곰',
 '곱',
 '곳',
 '공',
 '과',
 '관',
 '광',
 '괜',
 '괴',
 '굉',
 '교',
 '구',
 '국',
 '군',
 '굴',
 '굽',
 '궁',
 '궂',
 '권',
 '귀',
 '귄',
 '규',
 '그',
 '극',
 '근',
 '글',
 '금',
 '급',
 '긋',
 '긍',
 '기',
 '긴',
 '길',
 '깊',
 '까',
 '깎',
 '깐',
 '깔',
 '깜',
 '깨',
 '깼',
 '꺼',
 '꺾',
 '껍',
 '껏',
 '껐',
 '께',
 '껴',
 '꼬',
 '꼭',
 '꼼',
 '꼿',
 '꽃',
 '꽝',
 '꽤',
 '꾀',
 '꾸',
 '꾼',
 '꿀',
 '꿈',
 '뀌',
 '끄',
 '끈',
 '끌',
 '끝',
 '끼',
 '낀',
 '낄',
 '낌',
 '나',
 '낙',
 '난',
 '날',
 '낡',
 '남',
 '납',
 '났',
 '낭',
 '낮',
 '내',
 '낼',
 '냅',
 '냈',
 '냐',
 '냥',
 '너',
 '넌',
 '널',
 '넓',
 '넘',
 '넣

In [10]:
len(chars) # 중복되는 문자 제거

852

In [11]:
# 캐릭터와 인덱스로 짝지어진 딕셔너리 생성
char_dict = dict((char, chars.index(char)) for char in chars) # 이 문법 외우기
char_dict

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 '<': 18,
 '>': 19,
 '?': 20,
 'b': 21,
 '使': 22,
 '儀': 23,
 '商': 24,
 '大': 25,
 '小': 26,
 '布': 27,
 '式': 28,
 '律': 29,
 '惑': 30,
 '星': 31,
 '的': 32,
 '紀': 33,
 '紅': 34,
 '詩': 35,
 '隊': 36,
 '가': 37,
 '각': 38,
 '간': 39,
 '갈': 40,
 '감': 41,
 '갑': 42,
 '값': 43,
 '갔': 44,
 '강': 45,
 '갖': 46,
 '같': 47,
 '개': 48,
 '객': 49,
 '거': 50,
 '걱': 51,
 '건': 52,
 '걷': 53,
 '걸': 54,
 '검': 55,
 '겁': 56,
 '것': 57,
 '게': 58,
 '겐': 59,
 '겠': 60,
 '겨': 61,
 '겪': 62,
 '견': 63,
 '결': 64,
 '겸': 65,
 '겹': 66,
 '겼': 67,
 '경': 68,
 '곁': 69,
 '계': 70,
 '고': 71,
 '곡': 72,
 '곤': 73,
 '곧': 74,
 '골': 75,
 '곰': 76,
 '곱': 77,
 '곳': 78,
 '공': 79,
 '과': 80,
 '관': 81,
 '광': 82,
 '괜': 83,
 '괴': 84,
 '굉': 85,
 '교': 86,
 '구': 87,
 '국': 88,
 '군': 89,
 '굴': 90,
 '굽': 91,
 '궁': 92,
 '궂': 93,
 '권': 94,
 '귀': 95,
 '귄': 96,
 '규': 97,
 '그': 98,
 '극': 99,
 '근': 100

In [12]:
# 텍스트를 처리하는 방법에는 단어 기반과 캐릭터 기반 존재, 여기서는 한 글자 단위로 하는 캐릭터 방식, 전체 문서에서 고유한 글자 집합을 딕셔너리로 생성, 여기서 만든 인덱스 번호를 다시 원핫인코딩 변환,
# 이렇게 글자를 신경망이 처리할 수 있는 숫자 벡터로 만들어야 함

In [24]:
input_sentence_list

SyntaxError: invalid syntax (<ipython-input-24-a26e71130479>, line 1)

In [35]:
char_dict[output_char_list[0]]

497

In [41]:
# 문자를 원핫인코딩으로 설정하기 위해 초기화
# output은 1글자
x_train = np.zeros((len(input_sentence_list),input_len, len(chars)),dtype=np.int8) # batch_size, time_step, feature
y_train = np.zeros((len(input_sentence_list), len(chars)),dtype=np.int8) # batch_size, feature

# 문장에서 각 캐릭터 위치에 1로 설정하고 나머지는 모두 0
for i, sentence in enumerate(input_sentence_list):
    for j, char in enumerate(sentence): # 각 캐릭터 별로 분리
        x_train[i,j,char_dict[char]]=1
    
    y_train[i][char_dict[output_char_list[i]]]=1 # 이중리스트를 [,]로 표현하기도 한다

In [39]:
y_train[0][497]

1

In [42]:
x_train.shape # (전체 문자 개수, 입력 길이, 고유한 문자 개수)
y_train.shape # (전체 문자 개수, 고유한 문자 개수)

(45743, 852)

이전 IMDb 영화 리뷰 분류에서는 한 문장을 하나의 벡터에 모두 표현하는 Bow(Bag-of-words)를 사용, 이번에는 하나의 문장을 각 글자로 나누고 이를 원핫인코딩으로 변환

In [16]:
# 훈련셋 출력
x_train

array([[[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [44]:
# 훈련셋 라벨 출력
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

# 모델 생성

In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
def build_model():
    model = Sequential()
    model.add(LSTM(128, input_shape = (input_len, len(chars))))
    model.add(Dense(len(chars), activation='softmax'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    return model

iniput_shape에서 input_len은 입력의 글자 수로 순환신경망의 타임 스텝, 중요한 점은 신경망의 입력이 'input_len * len(chars)'의 크기가 아니라는 것, 입력 노드의 개수는 len(chars)이고, 이 하나의 신경망 모델을 input_len의 타임 스텝만큼 반복하여 돌림

# 훈련 및 테스트

In [46]:
# 시작 문장이 주어지면 이어지는 텍스트 생성
def generate_text(model, start_text):
    print('\n-시작 텍스트:')
    print(start_text)
    print()
    print('-텍스트 생성:')

    # 시작 텍스트로 텍스트 생성 초기화
    generated_text = start_text
    sys.stdout.write(generated_text) # 문자열 출력하기

    # 시작 텍스트에 이어지는 200개의 글자 생성
    for i in range(200):
        # 텍스트 벡터 초기화
        # 3차원
        text_vector = np.zeros((1,input_len, len(chars)))
        # 텍스트를 원핫인코딩으로 변경
        for j, char in enumerate(generated_text):
            text_vector[0,j,char_dict[char]]=1.
        # 다음 글자를 예측
        logits = model.predict(text_vector, verbose=0)
        char_index = np.argmax(logits[0])
        char_next = chars[char_index]

        # 다음 글자를 추가
        generated_text +=char_next

        # 첫 번째 글자를 삭제하여 전체 입력 길이 유지
        generated_text = generated_text[1:]

        # 캐릭터 글자 출력
        sys.stdout.write(char_next)
        sys.stdout.flush()


시작 문장이 주어지면 거기에 이어지는 200개의 글자를 생성합니다. 입력 문장의 바로 다음 글자를 예측하고, 이 글자가 다시 입력 문장에 추가됩니다. 동시에 처음 글자는 삭제되어 전체 입력 문장의 길이를 유지합니다.

동해물과백두산 -> 이<br>
해물과백두산이 -> 마<br>
물과백두산이마 -> 르
<br>
<br>
<br>

In [47]:
# 랜덤 시드 선택
random.seed(1212)

# 시드 텍스트의 위치를 랜덤하게 결정
start_index = random.randint(0, len(text)-input_len -1)

# 모델 생성
model = build_model()

# 훈련 반복
for epoch in range(0,50):
    print('Total Epoch:',epoch+1)

    # 모델 훈련
    model.fit(x_train,y_train,epochs=1, batch_size=128)

    # 텍스트 생성
    start_text = text[start_index: start_index + input_len]
    generate_text(model,start_text)
    print('\n')

Total Epoch: 1
Epoch 1/1
45743/45743 [==============================] - 13s 290us/step - loss: 4.3850

-시작 텍스트:
요?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지

-텍스트 생성:
요?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지이다다다다다다다다다다다다다다다다다다다다다다

Total Epoch: 2
Epoch 1/1
45743/45743 [==============================] - 13s 281us/step - loss: 3.8093

-시작 텍스트:
요?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지

-텍스트 생성:
요?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지그는그이다...............................................................................................................................................................................................

Total Epoch: 3
Epoch 1/1
45743/45743 [==============================] - 13s 282us/step - loss: 3.4925

-시작 텍스트:
요?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지

-텍스트 생성:
요?"
 어린 왕자가 그에게 물었다.
 "잊기 위해서지그는그이있었다.
"그그이있는그이있었다.
"그그이있는그이있었다.
"그그이있는그이있었다.
"그그이있는그이있었다.
"그그이있는그이있었다.
"그그이있는그이있었다.
"그그이있는그이있었다.
"그그이있는그이있었다.
"그그이있는그이있었다.
"그그이

Total Epoch: 4
Epoch 1/1
45743/45743 [==============================] - 13s 286us/step - loss: 3

In [49]:
# 모델 저장
model.save('D:/STUDY/model/text_generation_model.h5')

# 텍스트 생성

In [51]:
# 저장된 모델 파일 로드
model = models.load_model('D:/STUDY/model/text_generation_model.h5')


In [52]:
# 새로운 문장으로 텍스트 생성
start_text = '사막이 아름다운 것은 그것이 어딘가에 샘을 감추고 있기 때문이지'
# 길때는 자르고
if len(start_text) > input_len:
    start_text = start_text[:input_len]
# 짧을 때는 공백으로 채우고
if len(start_text) < input_len:
    start_text = start_text+ ' '*(input_len-len(start_text))

generate_text(model,start_text)


-시작 텍스트:
사막이 아름다운 것은 그것이 어딘가에 샘을 감추고 있기

-텍스트 생성:
사막이 아름다운 것은 그것이 어딘가에 샘을 감추고 있기때문이지....."
어린왕자가말했다.
"그래......이야기를좋이야......"
그는대답하지않았다.그가말했다.
"나는그들을위해같은해야해야."
어린왕자가말했다.
그러자그꽃은아무것도이라고가로고다다다는것을그리고있는이야하는다.어린왕자가무대했다.
"안다.난말에,"
"그리고그는이렇게말하지

In [ ]:
# 파라미터의 크기가 작고 데이터가 적기 때문에 문장의 품질이 그리 좋지 않습니다. 하지만 GPT2같은 모델은 진짜 사람이 쓴 것과 구별이 힘들 정도로 뛰어난 텍스트를 생성합니다. 